# Concepts

## DataArray and Dataset coord handling

This section describes details about how coords (and masks) of datasets and data arrays behave when slicing, combining, or inserting.

In [ ]:
import numpy as np
import scipp as sc

x = sc.Variable(dims=['x'], values=[1,2,3,4])
da = sc.DataArray(data=x,
                  coords={'x':x},
                  masks={'x':sc.less(x, 2 * sc.units.one)})
ds = sc.Dataset({'a':da})

Consider a data array `da` and a dataset `ds` with an aligned coord and an aligned mask.
The following conditions must hold:

In [ ]:
assert 'x' in da['x', 0:1].coords # range slice preserves coord
assert 'x' in da['x', 0:1].masks # range slice preserves coord
assert 'x' in da['x', 0].attrs # point slice converts coord to attr
assert 'x' not in da['x', 0].coords
assert 'x' in da['x', 0].attrs
assert 'x' in da['x', 0].masks # point slice preserves masks as aligned

In [ ]:
assert sc.is_equal(ds['a']['x', 0:1], ds['x', 0:1]['a'])
assert sc.is_equal(ds['a']['x', 0], ds['x', 0]['a'])

In [ ]:
assert 'x' in ds['a'].coords
assert 'x' in ds['x', 0:1].coords
assert 'x' not in ds['x', 0].coords # cannot have attr (unaligned coord) in dataset
assert 'x' in ds['x', 0:1]['a'].coords
assert 'x' in ds['a']['x', 0].attrs
assert 'x' in ds['x', 0]['a'].attrs

assert 'x' in ds['a'].masks
assert 'x' in ds['x', 0:1]['a'].masks
assert 'x' in ds['a']['x', 0].masks
assert 'x' in ds['x', 0]['a'].masks

In operations, coords are compared:

In [ ]:
try:
    ok = da['x', 0:1] + da['x', 1:2]
except:
    ok = False
assert not ok

Mismatching attrs ("unaligned coords") are dropped:

In [ ]:
assert sc.is_equal(da + da['x', 1], da + da['x', 1].data)

Masks are ORed, there is no concept of "unaligned masks":

In [ ]:
assert not sc.is_equal(da + da['x', 0], da + da['x', 0].data)

A missing attr is interpreted as mismatch to ensure that:

In [ ]:
a = da['x', 0]
b = da['x', 1]
c = da['x', 2]
assert sc.is_equal(a + (b + c), (a + b) + c)

Insertion order does not matter for attrs:

In [ ]:
a = da.copy()
a.attrs['attr'] = 1.0 * sc.units.m
b = da.copy()
b.attrs['attr'] = 2.0 * sc.units.m
ds1 = sc.Dataset()
ds2 = sc.Dataset()
ds1['a'] = a
ds1['b'] = b
ds2['b'] = b
ds2['a'] = a
assert sc.is_equal(ds1, ds2)

Insert into dataset with mismatching attrs drops attr:

In [ ]:
ds = sc.Dataset()
ds.coords['x'] = x['x', 0]
ds['a'] = da['x', 1] # Drops 'x' from 'a' 
assert sc.is_equal(ds.coords['x'], ds['a'].coords['x']) # shadowing is NOT supported

Masks of dataset items are independent:

In [ ]:
ds = sc.Dataset()
masked1 = da.copy()
masked1.masks['x'] = sc.less(x, 1 * sc.units.one)
masked2 = da.copy()
masked2.masks['x'] = sc.less(x, 2 * sc.units.one)
assert not sc.is_equal(masked1, masked2)
ds['a'] = masked1
ds['b'] = masked2
assert not sc.is_equal(ds['a'].masks['x'], ds['b'].masks['x'])

If there is no coord it is preserved for all items.
Adding a coord later fails to prevent shadowing:

In [ ]:
ds = sc.Dataset()
ds['a'] = da['x', 0]
ds['b'] = da['x', 1]
assert 'x' not in ds.coords
assert 'x' in ds['a'].attrs
assert 'x' in ds['b'].attrs
try:
    ds.coords['x'] = x['x', 0] # would introduce shadowing
except:
    ok = True
else:
    ok = False
assert ok

In [ ]:
edges = sc.Variable(dims=['x'], values=[1,2,3,4,5])
da.coords['x'] = edges
assert sc.is_equal(sc.concatenate(da['x', :2], da['x', 2:], 'x'), da)
assert sc.is_equal(sc.concatenate(da['x', 0], da['x', 1], 'x'), da['x', 0:2])
assert sc.is_equal(sc.concatenate(da['x', :-1], da['x', -1], 'x'), da)
da_yx = sc.concatenate(da['x', :2], da['x', 2:], 'y') # create 2-D coord
assert sc.is_equal(da_yx.coords['x'], sc.concatenate(da.coords['x']['x', :3], da.coords['x']['x', 2:], 'y'))

2-D coords for a dimension prevent operations between slices that are not along that dimension:

In [ ]:
da_2d = sc.DataArray(
    data=sc.Variable(['y', 'x'], shape=[2, 2]),
    coords={
        'x':sc.Variable(['y', 'x'], values=np.array([[1, 2], [3, 4]])),
        'y':sc.Variable(['y'], values=[3, 4])})

da_2d['x', 0] + da_2d['x', 1] # Same as with 1-D coord: x-coord differs but not aligned due to slice.
try:
    # 'y' sliced, so 'x' coord is aligned and yields different values from slices of 2-D coord.
    da_2d['y', 0] + da_2d['y', 1] 
except RuntimeError:
    ok = False
else:
    ok = True
assert not ok

`coords` always refers to (aligned) coords in dataset, cannot add or erase via item:

In [ ]:
try:
    ds['a'].coords['fail'] = 1.0 * sc.units.m
except RuntimeError:
    ok = False
else:
    ok = True
assert not ok

In [ ]:
ds.coords['xx'] = 1.0 * sc.units.m
assert 'xx' in ds['a'].coords
try:
    del ds['a'].coords['xx']
except RuntimeError:
    ok = False
else:
    ok = True
assert not ok

`meta` contains dataset coords as well as item attrs, cannot add or erase, since ambiguous:

In [ ]:
try:
    ds['a'].meta['fail'] = 1.0 * sc.units.m
except RuntimeError:
    ok = False
else:
    ok = True
assert not ok

In [ ]:
ds['a'].attrs['attr'] = 1.0 * sc.units.m
assert 'attr' in ds['a'].meta
try:
    del ds['a'].meta['attr']
except RuntimeError:
    ok = False
else:
    ok = True
assert not ok